In [1]:
!pip install python-docx
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.9 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import numpy as np
import torch
from torch import nn
from sklearn.utils.class_weight import compute_class_weight
from datasets import load_dataset
from transformers import (
    LongformerTokenizer,
    LongformerForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from transformers.trainer_pt_utils import IterableDatasetShard

# --- Constants ---
MODEL_NAME = "allenai/longformer-base-4096"
DATA_PATH = '/content/drive/MyDrive/DEEP_ML_Project-main/data/preprocessed/summary_results.csv'  # Columns: text, ICD9_4019, ICD9_4280, ...
OUTPUT_DIR = '/content/drive/MyDrive/DEEP_ML_Project-main/data/preprocessed/longformer_icd9_finetuned'
MAX_LENGTH = 4096  # Adjust based on note length (4096 max)

# --- Load Data ---
dataset = load_dataset("csv", data_files=DATA_PATH, split="train")

# Split train/validation
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_data = dataset["train"]
val_data = dataset["test"]

# --- Tokenization ---
tokenizer = LongformerTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["summary_snippet_clean"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH
    )

train_data = train_data.map(tokenize, batched=True, batch_size=8)
val_data = val_data.map(tokenize, batched=True, batch_size=8)

# Convert ICD9 columns to multi-hot labels
icd9_columns = [col for col in train_data.column_names if col.startswith("ICD9")]
train_data = train_data.map(lambda x: {"labels": [x[col] for col in icd9_columns]})
val_data = val_data.map(lambda x: {"labels": [x[col] for col in icd9_columns]})

# --- Class Weighting ---
all_labels = np.array(train_data["labels"])
class_weights = compute_class_weight(
    "balanced",
    classes=np.arange(len(icd9_columns)),
    y=all_labels.argmax(axis=1)
)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# --- Model Setup ---
model = LongformerForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(icd9_columns),
    problem_type="multi_label_classification"
)

# Optional: LoRA for efficient fine-tuning
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # ~1% of parameters trained

# --- Custom Loss ---
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").float()  # Remove labels from inputs
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.BCEWithLogitsLoss(weight=class_weights.to(labels.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# --- Training Args ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    fp16=True,
    eval_strategy="steps",  # Correct parameter name
    eval_steps=500,
    save_strategy="steps",  # Also updated from save_steps
    save_steps=1000,
    logging_steps=100,
    report_to="wandb",
    dataloader_num_workers=4,
    load_best_model_at_end=True,
)

# --- Train ---
# Then use it as before:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to {OUTPUT_DIR}")

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: goldwolf228 (goldwolf228-university-of-north-carolina-at-charlotte) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Initializing global attention on CLS token...


Step,Training Loss,Validation Loss


In [ ]:
import torch
from transformers import LongformerTokenizer, LongformerForSequenceClassification

model_path = "./longformer_icd9_finetuned"
tokenizer = LongformerTokenizer.from_pretrained(model_path)
model = LongformerForSequenceClassification.from_pretrained(model_path)

def predict_icd9(text, threshold=0.3):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).squeeze()
    return [model.config.id2label[i] for i, prob in enumerate(probs) if prob > threshold]

# Example
text = "Patient with hypertension and congestive heart failure..."
print(predict_icd9(text))  # Output: ["ICD9_4019", "ICD9_4280"]